# Transform Xenium outputs to spatialdata object for subsequent loading to scportrait

In [ ]:
from spatialdata_io import xenium # note this requires version 0.2.0! 0.3.0 does not work!
import scportrait

/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, 

In [2]:
input_dir = "../raw_input_data"
output_file = "../processed_data/raw_xenium.sdata.zarr"

In [3]:
sdata = xenium(
    path=input_dir,
    n_jobs=8,
    cells_boundaries=False,
    nucleus_boundaries=False,
    morphology_focus=True,
    aligned_images=True,
    cells_as_circles = False,
    cells_table=True,
)

INFO     reading ../raw_input_data/cell_feature_matrix.h5                                                          
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           


/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/tifffile/tifffile.py:8847: UserWarning: <tifffile.TiffPage 0 @16> reading array from closed file
  warnings.warn(
/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/tifffile/tifffile.py:8847: UserWarning: <tifffile.TiffPage 0 @16> reading array from closed file
  warnings.warn(
/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/tifffile/tifffile.py:8847: UserWarning: <tifffile.TiffPage 0 @16> reading array from closed file
  warnings.warn(
/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/tifffile/tifffile.py:8847: UserWarning: <tifffile.TiffPage 0 @16> reading array from closed file
  warnings.warn(


In [4]:
# convert table index to str as this is what anndata prefers to work with
sdata["table"].obs.index = sdata["table"].obs.index.astype('str')

In [5]:
sdata.write(output_file, overwrite = True)

INFO     The Zarr backing store has been changed from None the new file path:                                      
         ../processed_data/raw_xenium.sdata.zarr                                                                   


In [6]:
# convert HE image to rgb
sdata.set_channel_names("he_image", ["r", "g", "b"])
sdata.write_channel_names("he_image") #update the zarr store with the new channel names

/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/spatialdata/_core/_elements.py:71: UserWarning: Key `he_image` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


## rescale input image to 0.1 and 99.9%-tile

In [7]:
scportrait.tools.sdata.pp.percentile_normalize_image(sdata, 
                                                     image_name = "morphology_focus",
                                                     lower_percentile = 0.1, 
                                                     upper_percentile = 99.9, 
                                                     rescaled_image_name = "morphology_focus_rescaled")

version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: /Users/sophia/Documents/GitHub/scportrait_manuscript/input_data/Xenium_ovarian_cancer/processed_data/raw_xenium.sdata.zarr
├── Images
│     ├── 'he_image': DataTree[cyx] (3, 43993, 30918), (3, 21996, 15459), (3, 10998, 7729), (3, 5499, 3864), (3, 2749, 1932)
│     ├── 'morphology_focus': DataTree[cyx] (4, 37631, 54089), (4, 18815, 27044), (4, 9407, 13522), (4, 4703, 6761), (4, 2351, 3380)
│     └── 'morphology_focus_rescaled': DataTree[cyx] (4, 37631, 54089), (4, 18815, 27044), (4, 4703, 6761), (4, 587, 845)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (37631, 54089), (18815, 27044), (9407, 13522), (4703, 6761), (2351, 3380)
│     └── 'nucleus_labels': DataTree[yx] (37631, 54089), (18815, 27044), (9407, 13522), (4703, 6761), (2351, 3380)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
└── Tables
      └── 'table': AnnData (407124, 5101)
with coordinate systems:
    ▸ 'global', with elements:
     

In [9]:
sdata["table"]

AnnData object with n_obs × n_vars = 407124 × 5101
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'cell_labels'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial'